In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os

base_dir = '/kaggle/input/rice-data-cse475/Original'
subdirs = sorted(os.listdir(base_dir))
print(f'Found {len(subdirs)} class-folders, e.g.:', subdirs[:5])

# Count files in each
for d in subdirs[:5]:
    n = len(os.listdir(os.path.join(base_dir, d)))
    print(f'  {d}: {n} files')

Found 1 class-folders, e.g.: ['Original']
  Original: 38 files


In [4]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

# 1. Paths & hyper-parameters
base_dir    = '/kaggle/input/rice-data-cse475/Original'
batch_size  = 32
num_epochs  = 20
num_classes = 38
device      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


    

In [7]:
# 2. Data transforms
train_tf = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
val_tf = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

In [9]:
# 3. Dataset & split
full_dataset = datasets.ImageFolder(base_dir, transform=train_tf)
train_size   = int(0.8 * len(full_dataset))
val_size     = len(full_dataset) - train_size
train_ds, val_ds = random_split(
    full_dataset, 
    [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)
# Make sure val uses the val transforms
val_ds.dataset.transform = val_tf
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=4)
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=4)


In [10]:
# 4. Model setup
model = models.resnet50(pretrained=True)
# replace final layer
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
# two-stage optimizer: head first
opt_head = optim.Adam(model.fc.parameters(), lr=1e-3, weight_decay=1e-4)
# then full fine-tune
opt_full = optim.SGD(model.parameters(), lr=1e-4, momentum=0.9, weight_decay=1e-4)
sched    = optim.lr_scheduler.StepLR(opt_full, step_size=7, gamma=0.1)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 204MB/s]


In [12]:
# 5. Training & validation loops
def run_epoch(loader, model, criterion, optimizer=None):
    is_train = optimizer is not None
    loop = tqdm(loader, desc='Train' if is_train else ' Val ')
    running_loss, running_corrects = 0.0, 0
    for x, y in loop:
        x, y = x.to(device), y.to(device)
        with torch.set_grad_enabled(is_train):
            logits = model(x)
            loss   = criterion(logits, y)
            preds  = torch.argmax(logits, dim=1)

            if is_train:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        running_loss    += loss.item() * x.size(0)
        running_corrects+= (preds == y).sum().item()

    epoch_loss = running_loss / len(loader.dataset)
    epoch_acc  = running_corrects / len(loader.dataset)
    return epoch_loss, epoch_acc

In [13]:
# 6. Stage 1: Train head only
print("\n=== Stage 1: training head ===")
for epoch in range(3):
    tl, ta = run_epoch(train_loader, model, criterion, opt_head)
    vl, va = run_epoch(val_loader,   model, criterion, None)
    print(f"[Head] Epoch {epoch+1}/3  train_loss={tl:.3f} train_acc={ta:.3f}  val_loss={vl:.3f} val_acc={va:.3f}")

# 7. Stage 2: Fine-tune all layers
print("\n=== Stage 2: fine-tuning full model ===")
for epoch in range(num_epochs):
    tl, ta = run_epoch(train_loader, model, criterion, opt_full)
    sched.step()
    vl, va = run_epoch(val_loader,   model, criterion, None)
    print(f"[Full] Epoch {epoch+1}/{num_epochs}  train_loss={tl:.3f} train_acc={ta:.3f}  val_loss={vl:.3f} val_acc={va:.3f}")

# 8. Save best model
torch.save(model.state_dict(), 'rice_resnet50_finetuned.pt')


=== Stage 1: training head ===


 Val : 100%|██████████| 119/119 [00:16<00:00,  7.21it/s]


[Head] Epoch 1/3  train_loss=0.013 train_acc=0.998  val_loss=0.000 val_acc=1.000


 Val : 100%|██████████| 119/119 [00:13<00:00,  8.58it/s]


[Head] Epoch 2/3  train_loss=0.000 train_acc=1.000  val_loss=0.000 val_acc=1.000


...

[Full] Epoch 18/20  train_loss=0.000 train_acc=1.000  val_loss=0.000 val_acc=1.000


 Val : 100%|██████████| 119/119 [00:13<00:00,  8.51it/s]


[Full] Epoch 19/20  train_loss=0.000 train_acc=1.000  val_loss=0.000 val_acc=1.000


 Val : 100%|██████████| 119/119 [00:14<00:00,  8.45it/s]


[Full] Epoch 20/20  train_loss=0.000 train_acc=1.000  val_loss=0.000 val_acc=1.000
